In [66]:
import pandas as pd
import re

# Cleaning the 'races' file

In [51]:
races = pd.read_csv('../data/races.csv', index_col = 0)

In [52]:
races.head(10)

,0,1,0.1,1.1,2
0,19,NOV,Valencia MotoGP™ Official Test,Circuit Ricardo Tormo,SPAIN
1,25,NOV,Jerez MotoGP™ Official Test,Circuito de Jerez - Angel Nieto,SPAIN
2,7,FEB,Sepang MotoGP™ Official Test,Sepang International Circuit,MALAYSIA
3,19,FEB,Jerez Moto2™-Moto3™ Test,Circuito de Jerez - Angel Nieto,SPAIN
4,22,FEB,Qatar MotoGP™ Test,Losail International Circuit,QATAR
5,28,FEB,Qatar Moto2™-Moto3™ Test,Losail International Circuit,QATAR
6,8,MAR,1 - Grand Prix of Qatar,Losail International Circuit,QATAR
7,22,MAR,2 - OR Thailand Grand Prix,TT Circuit Assen,NETHERLANDS
8,5,APR,3 - Red Bull Grand Prix of The Americas,Circuit Of The Americas,UNITED STATES
9,19,APR,4 - Gran Premio Motul de la República Argentina,Termas de Río Hondo,ARGENTINA


## Renaming columns

In [53]:
races.rename(columns = {'0':'Day', '1':'Month', '0.1':'Race', '1.1':'Circuit', '2':'Country'}, inplace = True)

In [54]:
races.head()

,Day,Month,Race,Circuit,Country
0,19,NOV,Valencia MotoGP™ Official Test,Circuit Ricardo Tormo,SPAIN
1,25,NOV,Jerez MotoGP™ Official Test,Circuito de Jerez - Angel Nieto,SPAIN
2,7,FEB,Sepang MotoGP™ Official Test,Sepang International Circuit,MALAYSIA
3,19,FEB,Jerez Moto2™-Moto3™ Test,Circuito de Jerez - Angel Nieto,SPAIN
4,22,FEB,Qatar MotoGP™ Test,Losail International Circuit,QATAR


## Removing 'Tests'

For this project, I will focus only in scoring races which are called 'Grand Prix'.

In [55]:
# The new dataframe should not include races with 'Test' word in the 'Race' column.
races = races.loc[races['Race'].str.contains('Test') == False]

In [56]:
races.head()

,Day,Month,Race,Circuit,Country
6,8,MAR,1 - Grand Prix of Qatar,Losail International Circuit,QATAR
7,22,MAR,2 - OR Thailand Grand Prix,TT Circuit Assen,NETHERLANDS
8,5,APR,3 - Red Bull Grand Prix of The Americas,Circuit Of The Americas,UNITED STATES
9,19,APR,4 - Gran Premio Motul de la República Argentina,Termas de Río Hondo,ARGENTINA
10,3,MAY,5 - Gran Premio Red Bull de España,Circuito de Jerez - Angel Nieto,SPAIN


# Updating dates

Now we removed the tests, all the dates are for same year 2020. I will create a new column with 'Date' as a DateTime

In [57]:
races['Date'] = pd.to_datetime((races['Day'].astype(str)+races['Month'] + '2020'), errors = 'coerce', infer_datetime_format=True)

In [62]:
races = races[['Date', 'Race', 'Circuit', 'Country']].reset_index(drop=True)

In [63]:
races.head()

,Date,Race,Circuit,Country
0,2020-03-08,1 - Grand Prix of Qatar,Losail International Circuit,QATAR
1,2020-03-22,2 - OR Thailand Grand Prix,TT Circuit Assen,NETHERLANDS
2,2020-04-05,3 - Red Bull Grand Prix of The Americas,Circuit Of The Americas,UNITED STATES
3,2020-04-19,4 - Gran Premio Motul de la República Argentina,Termas de Río Hondo,ARGENTINA
4,2020-05-03,5 - Gran Premio Red Bull de España,Circuito de Jerez - Angel Nieto,SPAIN


In [73]:
races.dtypes

Date       datetime64[ns]
Race               object
Circuit            object
Country            object
dtype: object

# Removing numbers on Race names

As they are already ordered, we can remove the numbers at the beginning of the race's name.

In [68]:
races['Race'] = races['Race'].apply(lambda x: re.sub(r'\d+ - ','',x))

In [69]:
races.head()

,Date,Race,Circuit,Country
0,2020-03-08,Grand Prix of Qatar,Losail International Circuit,QATAR
1,2020-03-22,OR Thailand Grand Prix,TT Circuit Assen,NETHERLANDS
2,2020-04-05,Red Bull Grand Prix of The Americas,Circuit Of The Americas,UNITED STATES
3,2020-04-19,Gran Premio Motul de la República Argentina,Termas de Río Hondo,ARGENTINA
4,2020-05-03,Gran Premio Red Bull de España,Circuito de Jerez - Angel Nieto,SPAIN


# Checking duplicates

Except for the country, the other columns should not have any duplicates.

In [83]:
races['Date'].duplicated(keep = False).any()

False

In [84]:
races['Race'].duplicated(keep = False).any()

False

In [85]:
races['Circuit'].duplicated(keep = False).any()

True

In [86]:
races.loc[races['Circuit'].duplicated(keep = False)]

,Date,Race,Circuit,Country
1,2020-03-22,OR Thailand Grand Prix,TT Circuit Assen,NETHERLANDS
9,2020-06-28,Motul TT Assen,TT Circuit Assen,NETHERLANDS


The calendar from motogp.com has an error, and so the data scraped. As is just one mistake (circuit and country for Thailand Grand Prix) I will correct it manually

In [93]:
races.at[1,'Circuit'] = 'Chang International Circuit'

In [94]:
races.at[1,'Country'] = 'THAILAND'

In [95]:
races.loc[races['Circuit'].duplicated(keep = False)]

,Date,Race,Circuit,Country
